<h2>Regression Model with Keras</h2>
<h3>Final peer graded Assignment</h3>


In [1]:
import pandas as pd 
import numpy as np

I will be using the same dataset provided in the Assignment.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head(3)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

let's see if we have any missing values

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data is Clean so lets bulid our model

In [6]:
concrete_data_cols = concrete_data.columns
concrete_data_cols

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [7]:
predictors = concrete_data[concrete_data_cols[concrete_data_cols != 'Strength']]  #ALL COLUMNS EXCEPT THE STENGTH COLUMN
target = concrete_data['Strength'] # THE STRENGTH COLUMN

In [8]:
predictors.head(3)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270


In [9]:
target.head(3)

0    79.99
1    61.89
2    40.27
Name: Strength, dtype: float64

In [10]:
predictors_norm = (predictors - predictors.mean())/ predictors.std()
predictors_norm

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597


Lets see the No of cols predictors have

In [11]:
n_cols = predictors_norm.shape[1]
n_cols

8

Lets import the Keras library and other packages from Keras library

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Now Lets bulild the neural network

In [13]:
def regression_model():
    model = Sequential()
    model.add(Dense(10,activation = 'relu', input_shape = (n_cols,)))
    model.add(Dense(10 , activation = 'relu'))
    model.add(Dense(10 , activation = 'relu')) 
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

## Train and Test Split from Sklearn


In [14]:
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


Splitting the data into a training and test sets by holding 30% of the data for testing



In [15]:
X_train, X_test , y_train , y_test = train_test_split(predictors, target, random_state = 4, test_size = 0.3) 

Let's call the function now to create our model.


In [16]:
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [17]:
model.fit(X_train, y_train , validation_split = 0.3 , epochs = 100 , verbose = 1)



Train on 504 samples, validate on 217 samples
Epoch 1/100


2022-08-26 14:28:44.492032: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-08-26 14:28:44.497391: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593900000 Hz
2022-08-26 14:28:44.498230: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558fb0f0ac80 executing computations on platform Host. Devices:
2022-08-26 14:28:44.498279: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-08-26 14:28:44.624866: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

504/504 [==============================] - 0s 741us/step - loss: 10859.2905 - val_loss: 5449.5992
Epoch 2/100
504/504 [==============================] - 0s 93us/step - loss: 3377.4212 - val_loss: 1403.5447
Epoch 3/100
504/504 [==============================] - 0s 153us/step - loss: 909.8634 - val_loss: 478.6119
Epoch 4/100
504/504 [==============================] - 0s 151us/step - loss: 521.4290 - val_loss: 456.5307
Epoch 5/100
504/504 [==============================] - 0s 150us/step - loss: 478.8605 - val_loss: 401.3813
Epoch 6/100
504/504 [==============================] - 0s 127us/step - loss: 427.5481 - val_loss: 368.2590
Epoch 7/100
504/504 [==============================] - 0s 120us/step - loss: 398.5122 - val_loss: 344.1332
Epoch 8/100
504/504 [==============================] - 0s 121us/step - loss: 367.4363 - val_loss: 322.0896
Epoch 9/100
504/504 [==============================] - 0s 119us/step - loss: 342.6244 - val_loss: 302.3208
Epoch 10/100
504/504 [=======================

let's evaluate the model on test data

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 39us/step


85.70382875609167

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
mean

85.7038279704333

In [21]:
stdv = np.std(mean_square_error)
stdv

0.0

Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [22]:

mean_squared_errors = list()
total_mean_square_errors = 50 
epochs = 100
for i in range(0, total_mean_square_errors):
    X_train, X_test , y_train , y_test = train_test_split(predictors, target, random_state = i, test_size = 0.3) 
    model.fit(X_train, y_train , validation_split = 0.3 , epochs = epochs , verbose = 0)
    MSE = model.evaluate(X_test, y_test, verbose = 0)
    print("MSE: "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

MSE: 1: 59.762725212813194
MSE: 2: 67.31382230493243
MSE: 3: 57.22955824102013
MSE: 4: 68.07677385645005
MSE: 5: 53.72172642680048
MSE: 6: 41.99550728659028
MSE: 7: 71.18550218501909
MSE: 8: 41.561378380241514
MSE: 9: 44.889203809225826
MSE: 10: 44.80584400757231
MSE: 11: 38.499568044174836
MSE: 12: 36.08049351343445
MSE: 13: 46.8458093686397
MSE: 14: 47.59498057936388
MSE: 15: 39.228468330161085
MSE: 16: 42.11022043073833
MSE: 17: 36.81035868326823
MSE: 18: 45.08198323913377
MSE: 19: 39.35237938840798
MSE: 20: 35.822714598819275
MSE: 21: 37.740694755875175
MSE: 22: 34.3715398844006
MSE: 23: 35.48464441145122
MSE: 24: 34.37374373006975
MSE: 25: 36.30769287652568
MSE: 26: 41.192432477636245
MSE: 27: 39.63175373015666
MSE: 28: 33.75120307403861
MSE: 29: 40.004363309989856
MSE: 30: 38.532861147883644
MSE: 31: 36.55242039702085
MSE: 32: 30.795799854118076
MSE: 33: 35.482664867512234
MSE: 34: 38.254198845921984
MSE: 35: 35.07660677517888
MSE: 36: 37.754942032897354
MSE: 37: 33.3595464576795

In [23]:
print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_square_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))



Below is the mean and standard deviation of 50 mean squared errors without normalized data. Total number of epochs for each training is: 100

Mean: 40.42746074159787
Standard Deviation: 9.486234906340554
